In [1]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/02_credit_ratings_with_mda.csv')

# Display the first 5 rows of the DataFrame
display(df.head())

,Rating,Name,Ticker,Rating Agency Name,Date,Sector,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,...,enterpriseValueMultiple,operatingCashFlowPerShare,operatingCashFlowSalesRatio,payablesTurnover,Rating_Merged,Rating_Encoded_Multiclass,Rating_Encoded_Binary,rating_date,year_qtr,md&a
0,A,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,11/27/2015,Consumer Durables,0.945894,0.426395,0.099690,44.203245,...,7.057088,15.565438,0.058638,3.906655,A,0,1,2015-11-27,2015_Q4,ABOUT WHIRLPOOL Whirlpool is the number one ma...
1,BBB,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,2/13/2014,Consumer Durables,1.033559,0.498234,0.203120,38.991156,...,6.460618,15.914250,0.067239,4.002846,BBB,5,1,2014-02-13,2014_Q1,ABOUT WHIRLPOOL Whirlpool Corporation (“Whirlp...
2,BBB,Whirlpool Corporation,WHR,Fitch Ratings,3/6/2015,Consumer Durables,0.963703,0.451505,0.122099,50.841385,...,10.491970,18.888889,0.074426,3.483510,BBB,5,1,2015-03-06,2015_Q1,ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS O...
3,BBB,Whirlpool Corporation,WHR,Fitch Ratings,6/15/2012,Consumer Durables,1.019851,0.510402,0.176116,41.161738,...,4.080741,6.901042,0.028394,4.581150,BBB,5,1,2012-06-15,2012_Q2,ABOUT WHIRLPOOL Whirlpool Corporation (“Whirlp...
4,BBB,Whirlpool Corporation,WHR,Standard & Poor's Ratings Services,10/24/2016,Consumer Durables,0.957844,0.495432,0.141608,47.761126,...,8.293505,15.808147,0.058065,3.857790,BBB,5,1,2016-10-24,2016_Q4,"ABOUT WHIRLPOOL Whirlpool Corporation (""Whirlp..."


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2029 entries, 0 to 2028
Data columns (total 37 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Rating                              2029 non-null   object 
 1   Name                                2029 non-null   object 
 2   Ticker                              2029 non-null   object 
 3   Rating Agency Name                  2029 non-null   object 
 4   Date                                2029 non-null   object 
 5   Sector                              2029 non-null   object 
 6   currentRatio                        2029 non-null   float64
 7   quickRatio                          2029 non-null   float64
 8   cashRatio                           2029 non-null   float64
 9   daysOfSalesOutstanding              2029 non-null   float64
 10  netProfitMargin                     2029 non-null   float64
 11  pretaxProfitMargin                  2029 no

In [3]:
df['Rating_Merged'].unique()

array(['A', 'BBB', 'AA', 'BB', 'B', 'CCC', 'CCC–', 'AA+'], dtype=object)

In [6]:
!pip install textblob nltk vaderSentiment textstat spacy readability readability-lxml
!pip install transformers torch
!python -m spacy download en_core_web_sm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.6 MB/s eta 0:00:00
  Created wheel for readability: filename=readability-0.3.2-py3-none-any.whl size=36383 sha256=1fa9a1b0ec15c95415ba3810336917c8aa48a0e39d8ce1c472924f8ba93a2bc8
  Stored in directory: /root/.cache/pip/wheels/f3/58/aa/53a1166ad162b65b2bebf0964c83b977b7091d57a10fe7e0b6
Successfully built readability
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 829.8 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [12]:
import pandas as pd
import numpy as np
import re
import nltk
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import textstat
from readability import Readability
import readability
from transformers import pipeline
import spacy

# Download required NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

class FinancialTextAnalyzer:
    def __init__(self):
        # Initialize analyzers
        self.vader = SentimentIntensityAnalyzer()
        self.nlp = spacy.load("en_core_web_sm")

        # Load financial word lists (you may need to create/customize these)
        self.risk_words = self._load_risk_words()
        self.fraud_words = self._load_fraud_words()
        self.safety_words = self._load_safety_words()
        self.certainty_words = self._load_certainty_words()
        self.uncertainty_words = self._load_uncertainty_words()
        self.litigious_words = self._load_litigious_words()

    def _load_risk_words(self):
        # Common financial risk-related words
        return {
            'risk', 'risks', 'uncertain', 'uncertainty', 'volatility', 'challenge',
            'challenges', 'threat', 'threats', 'vulnerable', 'exposure', 'exposed',
            'downturn', 'decline', 'decrease', 'loss', 'losses', 'adverse',
            'difficult', 'difficulty', 'pressure', 'pressures', 'weakness', 'weaknesses'
        }

    def _load_fraud_words(self):
        # Words related to fraud or misconduct
        return {
            'fraud', 'fraudulent', 'misconduct', 'irregularity', 'irregularities',
            'misstatement', 'misstatements', 'restatement', 'restatements',
            'investigation', 'investigations', 'allegation', 'allegations',
            'violation', 'violations', 'breach', 'breaches', 'non-compliance',
            'illegal', 'unlawful', 'improper', 'questionable'
        }

    def _load_safety_words(self):
        # Words indicating safety/stability
        return {
            'stable', 'stability', 'secure', 'security', 'strong', 'strength',
            'robust', 'reliable', 'dependable', 'consistent', 'conservative',
            'prudent', 'cautious', 'protected', 'hedged', 'insured', 'guaranteed',
            'solid', 'sound', 'healthy', 'resilient', 'resilience'
        }

    def _load_certainty_words(self):
        # Words indicating certainty
        return {
            'certain', 'definite', 'definitely', 'clear', 'clearly', 'obvious',
            'obviously', 'undoubtedly', 'without doubt', 'guarantee', 'guaranteed',
            'assure', 'assured', 'confident', 'confidence', 'will', 'shall',
            'must', 'certainty', 'positive', 'positively'
        }

    def _load_uncertainty_words(self):
        # Words indicating uncertainty
        return {
            'uncertain', 'uncertainty', 'may', 'might', 'could', 'possibly',
            'perhaps', 'maybe', 'potentially', 'likely', 'unlikely', 'probability',
            'probable', 'estimate', 'estimated', 'approximately', 'roughly',
            'about', 'around', 'subject to', 'depending on', 'contingent'
        }

    def _load_litigious_words(self):
        # Words related to litigation
        return {
            'litigation', 'lawsuit', 'lawsuits', 'legal', 'proceeding', 'proceedings',
            'claim', 'claims', 'dispute', 'disputes', 'arbitration', 'settlement',
            'settlements', 'judgment', 'judgments', 'court', 'appeal', 'appeals',
            'plaintiff', 'defendant', 'sue', 'sued', 'class action', 'alleged'
        }

    def calculate_positivity(self, text):
        """Calculate positivity score"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        vader_scores = self.vader.polarity_scores(text)
        blob_scores = TextBlob(text).sentiment

        # Weighted average of multiple sentiment measures
        positivity = (vader_scores['pos'] * 0.6 +
                      (blob_scores.polarity + 1) / 2 * 0.4)

        return round(positivity, 4)

    def calculate_negativity(self, text):
        """Calculate negativity score"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        vader_scores = self.vader.polarity_scores(text)
        blob_scores = TextBlob(text).sentiment

        # Weighted average
        negativity = (vader_scores['neg'] * 0.6 +
                      (1 - blob_scores.polarity) / 2 * 0.4)

        return round(negativity, 4)

    def calculate_litigiousness(self, text):
        """Calculate litigiousness score based on legal word frequency"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        words = text.lower().split()
        if len(words) == 0:
            return 0.0

        # Count litigious words
        litigious_count = sum(1 for word in words if word in self.litigious_words)

        # Normalize by text length (words)
        score = litigious_count / len(words)
        return round(score, 4)

    def calculate_polarity(self, text):
        """Calculate overall polarity (-1 to 1)"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        vader_scores = self.vader.polarity_scores(text)
        blob_polarity = TextBlob(text).sentiment.polarity

        # Combine scores
        polarity = (vader_scores['compound'] * 0.5 + blob_polarity * 0.5)
        return round(polarity, 4)

    def calculate_risk(self, text):
        """Calculate risk score based on risk-related words"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        words = text.lower().split()
        if len(words) == 0:
            return 0.0

        # Count risk words
        risk_count = sum(1 for word in words if word in self.risk_words)

        # Normalize by text length
        score = risk_count / len(words)
        return round(score, 4)

    def calculate_readability(self, text):
        """Calculate readability score (higher = more readable)"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        try:
            # Flesch Reading Ease score (higher = easier to read)
            flesch_score = textstat.flesch_reading_ease(text)

            # Normalize to 0-1 scale (assuming typical range 0-100)
            normalized_score = min(max(flesch_score / 100, 0), 1)

            return round(normalized_score, 4)
        except:
            return 0.0

    def calculate_fraud(self, text):
        """Calculate fraud-related language score"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        words = text.lower().split()
        if len(words) == 0:
            return 0.0

        # Count fraud-related words
        fraud_count = sum(1 for word in words if word in self.fraud_words)

        # Normalize by text length
        score = fraud_count / len(words)
        return round(score, 4)

    def calculate_safety(self, text):
        """Calculate safety/stability score"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        words = text.lower().split()
        if len(words) == 0:
            return 0.0

        # Count safety words
        safety_count = sum(1 for word in words if word in self.safety_words)

        # Normalize by text length
        score = safety_count / len(words)
        return round(score, 4)

    def calculate_certainty(self, text):
        """Calculate certainty language score"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        words = text.lower().split()
        if len(words) == 0:
            return 0.0

        # Count certainty words
        certainty_count = sum(1 for word in words if word in self.certainty_words)

        # Normalize by text length
        score = certainty_count / len(words)
        return round(score, 4)

    def calculate_uncertainty(self, text):
        """Calculate uncertainty language score"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        words = text.lower().split()
        if len(words) == 0:
            return 0.0

        # Count uncertainty words
        uncertainty_count = sum(1 for word in words if word in self.uncertainty_words)

        # Normalize by text length
        score = uncertainty_count / len(words)
        return round(score, 4)

    def calculate_sentiment(self, text):
        """Calculate overall sentiment score (-1 to 1)"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        vader_scores = self.vader.polarity_scores(text)
        return round(vader_scores['compound'], 4)

    def analyze_all_scores(self, text):
        """Calculate all 11 scores for a given text"""
        if pd.isna(text) or text.strip() == "":
            return {score: 0.0 for score in self.get_score_names()}

        return {
            'positivity': self.calculate_positivity(text),
            'negativity': self.calculate_negativity(text),
            'litigiousness': self.calculate_litigiousness(text),
            'polarity': self.calculate_polarity(text),
            'risk': self.calculate_risk(text),
            'readability': self.calculate_readability(text),
            'fraud': self.calculate_fraud(text),
            'safety': self.calculate_safety(text),
            'certainty': self.calculate_certainty(text),
            'uncertainty': self.calculate_uncertainty(text),
            'sentiment': self.calculate_sentiment(text)
        }

    def get_score_names(self):
        """Return list of all score names"""
        return [
            'positivity', 'negativity', 'litigiousness', 'polarity',
            'risk', 'readability', 'fraud', 'safety', 'certainty',
            'uncertainty', 'sentiment'
        ]

# Main processing function
def compute_nlp_scores(df, text_column='md&a'):
    """
    Compute NLP scores for MD&A text column

    Parameters:
    -----------
    df : pandas.DataFrame
        Input dataframe containing text data
    text_column : str
        Name of the column containing text data

    Returns:
    --------
    pandas.DataFrame with new columns added
    """

    analyzer = FinancialTextAnalyzer()

    # Create progress bar for large datasets
    from tqdm import tqdm
    tqdm.pandas()

    print("Computing NLP scores...")

    # Calculate all scores
    scores = df[text_column].progress_apply(
        lambda x: analyzer.analyze_all_scores(x) if pd.notna(x) and str(x).strip() != ""
        else {score: np.nan for score in analyzer.get_score_names()}
    )

    # Convert to DataFrame
    scores_df = pd.DataFrame(scores.tolist(), index=df.index)

    # Add prefix to column names
    scores_df = scores_df.add_prefix('nlp_')

    # Concatenate with original dataframe
    result_df = pd.concat([df, scores_df], axis=1)

    print(f"Added {len(scores_df.columns)} NLP score columns:")
    for col in scores_df.columns:
        print(f"  - {col}")

    return result_df

# Optional: Enhanced version with more sophisticated measures
def compute_nlp_scores_enhanced(df, text_column='md&a'):
    """
    Enhanced version with additional preprocessing and validation
    """

    # Preprocess text function
    def preprocess_text(text):
        if pd.isna(text):
            return ""

        text = str(text)
        # Basic cleaning
        text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
        text = re.sub(r'[^\w\s.,;:!?\-$%]', '', text)  # Remove special chars
        text = text.strip()
        return text

    print("Preprocessing text...")
    df['md&a_processed'] = df[text_column].apply(preprocess_text)

    # Initialize analyzer
    analyzer = FinancialTextAnalyzer()

    # Calculate scores
    scores_data = []
    print("Computing enhanced NLP scores...")

    for idx, text in tqdm(enumerate(df['md&a_processed']), total=len(df)):
        if text and len(text.split()) > 10:  # Only process meaningful text
            scores = analyzer.analyze_all_scores(text)
        else:
            scores = {score: np.nan for score in analyzer.get_score_names()}
        scores_data.append(scores)

    # Create scores dataframe
    scores_df = pd.DataFrame(scores_data, index=df.index)
    scores_df = scores_df.add_prefix('nlp_')

    # Add additional derived metrics
    if 'nlp_positivity' in scores_df.columns and 'nlp_negativity' in scores_df.columns:
        scores_df['nlp_sentiment_balance'] = scores_df['nlp_positivity'] - scores_df['nlp_negativity']
        scores_df['nlp_risk_safety_ratio'] = scores_df['nlp_risk'] / (scores_df['nlp_safety'] + 0.001)  # Avoid division by zero

    # Combine with original
    result_df = pd.concat([df, scores_df], axis=1)

    # Drop temporary processed column if desired
    result_df = result_df.drop(columns=['md&a_processed'])

    return result_df

ImportError: cannot import name 'Readability' from 'readability' (/usr/local/lib/python3.12/dist-packages/readability/__init__.py)

# Task
Calculate and add 11 NLP scores (positivity, negativity, litigiousness, polarity, risk, readability, fraud, safety, certainty, uncertainty, and sentiment) to the DataFrame `df` based on the text in the 'md&a' column, and then display the updated DataFrame.

## Correct NLP Library Imports

### Subtask:
Modify the imports in the `sPvvkOp_RVdz` cell to remove the problematic `readability` import, as `textstat` is already used for readability calculations.


**Reasoning**:
The previous code cell resulted in an ImportError related to the 'readability' library. To fix this, I will remove the problematic import statements from the `sPvvkOp_RVdz` cell, as `textstat` is already being used for readability calculations.



In [13]:
import pandas as pd
import numpy as np
import re
import nltk
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import textstat
from transformers import pipeline
import spacy

# Download required NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

class FinancialTextAnalyzer:
    def __init__(self):
        # Initialize analyzers
        self.vader = SentimentIntensityAnalyzer()
        self.nlp = spacy.load("en_core_web_sm")

        # Load financial word lists (you may need to create/customize these)
        self.risk_words = self._load_risk_words()
        self.fraud_words = self._load_fraud_words()
        self.safety_words = self._load_safety_words()
        self.certainty_words = self._load_certainty_words()
        self.uncertainty_words = self._load_uncertainty_words()
        self.litigious_words = self._load_litigious_words()

    def _load_risk_words(self):
        # Common financial risk-related words
        return {
            'risk', 'risks', 'uncertain', 'uncertainty', 'volatility', 'challenge',
            'challenges', 'threat', 'threats', 'vulnerable', 'exposure', 'exposed',
            'downturn', 'decline', 'decrease', 'loss', 'losses', 'adverse',
            'difficult', 'difficulty', 'pressure', 'pressures', 'weakness', 'weaknesses'
        }

    def _load_fraud_words(self):
        # Words related to fraud or misconduct
        return {
            'fraud', 'fraudulent', 'misconduct', 'irregularity', 'irregularities',
            'misstatement', 'misstatements', 'restatement', 'restatements',
            'investigation', 'investigations', 'allegation', 'allegations',
            'violation', 'violations', 'breach', 'breaches', 'non-compliance',
            'illegal', 'unlawful', 'improper', 'questionable'
        }

    def _load_safety_words(self):
        # Words indicating safety/stability
        return {
            'stable', 'stability', 'secure', 'security', 'strong', 'strength',
            'robust', 'reliable', 'dependable', 'consistent', 'conservative',
            'prudent', 'cautious', 'protected', 'hedged', 'insured', 'guaranteed',
            'solid', 'sound', 'healthy', 'resilient', 'resilience'
        }

    def _load_certainty_words(self):
        # Words indicating certainty
        return {
            'certain', 'definite', 'definitely', 'clear', 'clearly', 'obvious',
            'obviously', 'undoubtedly', 'without doubt', 'guarantee', 'guaranteed',
            'assure', 'assured', 'confident', 'confidence', 'will', 'shall',
            'must', 'certainty', 'positive', 'positively'
        }

    def _load_uncertainty_words(self):
        # Words indicating uncertainty
        return {
            'uncertain', 'uncertainty', 'may', 'might', 'could', 'possibly',
            'perhaps', 'maybe', 'potentially', 'likely', 'unlikely', 'probability',
            'probable', 'estimate', 'estimated', 'approximately', 'roughly',
            'about', 'around', 'subject to', 'depending on', 'contingent'
        }

    def _load_litigious_words(self):
        # Words related to litigation
        return {
            'litigation', 'lawsuit', 'lawsuits', 'legal', 'proceeding', 'proceedings',
            'claim', 'claims', 'dispute', 'disputes', 'arbitration', 'settlement',
            'settlements', 'judgment', 'judgments', 'court', 'appeal', 'appeals',
            'plaintiff', 'defendant', 'sue', 'sued', 'class action', 'alleged'
        }

    def calculate_positivity(self, text):
        """Calculate positivity score"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        vader_scores = self.vader.polarity_scores(text)
        blob_scores = TextBlob(text).sentiment

        # Weighted average of multiple sentiment measures
        positivity = (vader_scores['pos'] * 0.6 +
                      (blob_scores.polarity + 1) / 2 * 0.4)

        return round(positivity, 4)

    def calculate_negativity(self, text):
        """Calculate negativity score"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        vader_scores = self.vader.polarity_scores(text)
        blob_scores = TextBlob(text).sentiment

        # Weighted average
        negativity = (vader_scores['neg'] * 0.6 +
                      (1 - blob_scores.polarity) / 2 * 0.4)

        return round(negativity, 4)

    def calculate_litigiousness(self, text):
        """Calculate litigiousness score based on legal word frequency"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        words = text.lower().split()
        if len(words) == 0:
            return 0.0

        # Count litigious words
        litigious_count = sum(1 for word in words if word in self.litigious_words)

        # Normalize by text length (words)
        score = litigious_count / len(words)
        return round(score, 4)

    def calculate_polarity(self, text):
        """Calculate overall polarity (-1 to 1)"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        vader_scores = self.vader.polarity_scores(text)
        blob_polarity = TextBlob(text).sentiment.polarity

        # Combine scores
        polarity = (vader_scores['compound'] * 0.5 + blob_polarity * 0.5)
        return round(polarity, 4)

    def calculate_risk(self, text):
        """Calculate risk score based on risk-related words"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        words = text.lower().split()
        if len(words) == 0:
            return 0.0

        # Count risk words
        risk_count = sum(1 for word in words if word in self.risk_words)

        # Normalize by text length
        score = risk_count / len(words)
        return round(score, 4)

    def calculate_readability(self, text):
        """Calculate readability score (higher = more readable)"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        try:
            # Flesch Reading Ease score (higher = easier to read)
            flesch_score = textstat.flesch_reading_ease(text)

            # Normalize to 0-1 scale (assuming typical range 0-100)
            normalized_score = min(max(flesch_score / 100, 0), 1)

            return round(normalized_score, 4)
        except:
            return 0.0

    def calculate_fraud(self, text):
        """Calculate fraud-related language score"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        words = text.lower().split()
        if len(words) == 0:
            return 0.0

        # Count fraud-related words
        fraud_count = sum(1 for word in words if word in self.fraud_words)

        # Normalize by text length
        score = fraud_count / len(words)
        return round(score, 4)

    def calculate_safety(self, text):
        """Calculate safety/stability score"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        words = text.lower().split()
        if len(words) == 0:
            return 0.0

        # Count safety words
        safety_count = sum(1 for word in words if word in self.safety_words)

        # Normalize by text length
        score = safety_count / len(words)
        return round(score, 4)

    def calculate_certainty(self, text):
        """Calculate certainty language score"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        words = text.lower().split()
        if len(words) == 0:
            return 0.0

        # Count certainty words
        certainty_count = sum(1 for word in words if word in self.certainty_words)

        # Normalize by text length
        score = certainty_count / len(words)
        return round(score, 4)

    def calculate_uncertainty(self, text):
        """Calculate uncertainty language score"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        words = text.lower().split()
        if len(words) == 0:
            return 0.0

        # Count uncertainty words
        uncertainty_count = sum(1 for word in words if word in self.uncertainty_words)

        # Normalize by text length
        score = uncertainty_count / len(words)
        return round(score, 4)

    def calculate_sentiment(self, text):
        """Calculate overall sentiment score (-1 to 1)"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        vader_scores = self.vader.polarity_scores(text)
        return round(vader_scores['compound'], 4)

    def analyze_all_scores(self, text):
        """Calculate all 11 scores for a given text"""
        if pd.isna(text) or text.strip() == "":
            return {score: 0.0 for score in self.get_score_names()}

        return {
            'positivity': self.calculate_positivity(text),
            'negativity': self.calculate_negativity(text),
            'litigiousness': self.calculate_litigiousness(text),
            'polarity': self.calculate_polarity(text),
            'risk': self.calculate_risk(text),
            'readability': self.calculate_readability(text),
            'fraud': self.calculate_fraud(text),
            'safety': self.calculate_safety(text),
            'certainty': self.calculate_certainty(text),
            'uncertainty': self.calculate_uncertainty(text),
            'sentiment': self.calculate_sentiment(text)
        }

    def get_score_names(self):
        """Return list of all score names"""
        return [
            'positivity', 'negativity', 'litigiousness', 'polarity',
            'risk', 'readability', 'fraud', 'safety', 'certainty',
            'uncertainty', 'sentiment'
        ]

# Main processing function
def compute_nlp_scores(df, text_column='md&a'):
    """
    Compute NLP scores for MD&A text column

    Parameters:
    -----------
    df : pandas.DataFrame
        Input dataframe containing text data
    text_column : str
        Name of the column containing text data

    Returns:
    --------
    pandas.DataFrame with new columns added
    """

    analyzer = FinancialTextAnalyzer()

    # Create progress bar for large datasets
    from tqdm import tqdm
    tqdm.pandas()

    print("Computing NLP scores...")

    # Calculate all scores
    scores = df[text_column].progress_apply(
        lambda x: analyzer.analyze_all_scores(x) if pd.notna(x) and str(x).strip() != ""
        else {score: np.nan for score in analyzer.get_score_names()}
    )

    # Convert to DataFrame
    scores_df = pd.DataFrame(scores.tolist(), index=df.index)

    # Add prefix to column names
    scores_df = scores_df.add_prefix('nlp_')

    # Concatenate with original dataframe
    result_df = pd.concat([df, scores_df], axis=1)

    print(f"Added {len(scores_df.columns)} NLP score columns:")
    for col in scores_df.columns:
        print(f"  - {col}")

    return result_df

# Optional: Enhanced version with more sophisticated measures
def compute_nlp_scores_enhanced(df, text_column='md&a'):
    """
    Enhanced version with additional preprocessing and validation
    """

    # Preprocess text function
    def preprocess_text(text):
        if pd.isna(text):
            return ""

        text = str(text)
        # Basic cleaning
        text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
        text = re.sub(r'[^\w\s.,;:!?\-$%]', '', text)  # Remove special chars
        text = text.strip()
        return text

    print("Preprocessing text...")
    df['md&a_processed'] = df[text_column].apply(preprocess_text)

    # Initialize analyzer
    analyzer = FinancialTextAnalyzer()

    # Calculate scores
    scores_data = []
    print("Computing enhanced NLP scores...")

    for idx, text in tqdm(enumerate(df['md&a_processed']), total=len(df)):
        if text and len(text.split()) > 10:  # Only process meaningful text
            scores = analyzer.analyze_all_scores(text)
        else:
            scores = {score: np.nan for score in analyzer.get_score_names()}
        scores_data.append(scores)

    # Create scores dataframe
    scores_df = pd.DataFrame(scores_data, index=df.index)
    scores_df = scores_df.add_prefix('nlp_')

    # Add additional derived metrics
    if 'nlp_positivity' in scores_df.columns and 'nlp_negativity' in scores_df.columns:
        scores_df['nlp_sentiment_balance'] = scores_df['nlp_positivity'] - scores_df['nlp_negativity']
        scores_df['nlp_risk_safety_ratio'] = scores_df['nlp_risk'] / (scores_df['nlp_safety'] + 0.001)  # Avoid division by zero

    # Combine with original
    result_df = pd.concat([df, scores_df], axis=1)

    # Drop temporary processed column if desired
    result_df = result_df.drop(columns=['md&a_processed'])

    return result_df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**Reasoning**:
The previous attempt to run cell `sPvvkOp_RVdz` resulted in an `ImportError` due to the `readability` library. The provided code block correctly removes the problematic import statements, which resolves this issue and allows the NLP processing functions to run as intended.



In [9]:
import pandas as pd
import numpy as np
import re
import nltk
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import textstat
from transformers import pipeline
import spacy

# Download required NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

class FinancialTextAnalyzer:
    def __init__(self):
        # Initialize analyzers
        self.vader = SentimentIntensityAnalyzer()
        self.nlp = spacy.load("en_core_web_sm")

        # Load financial word lists (you may need to create/customize these)
        self.risk_words = self._load_risk_words()
        self.fraud_words = self._load_fraud_words()
        self.safety_words = self._load_safety_words()
        self.certainty_words = self._load_certainty_words()
        self.uncertainty_words = self._load_uncertainty_words()
        self.litigious_words = self._load_litigious_words()

    def _load_risk_words(self):
        # Common financial risk-related words
        return {
            'risk', 'risks', 'uncertain', 'uncertainty', 'volatility', 'challenge',
            'challenges', 'threat', 'threats', 'vulnerable', 'exposure', 'exposed',
            'downturn', 'decline', 'decrease', 'loss', 'losses', 'adverse',
            'difficult', 'difficulty', 'pressure', 'pressures', 'weakness', 'weaknesses'
        }

    def _load_fraud_words(self):
        # Words related to fraud or misconduct
        return {
            'fraud', 'fraudulent', 'misconduct', 'irregularity', 'irregularities',
            'misstatement', 'misstatements', 'restatement', 'restatements',
            'investigation', 'investigations', 'allegation', 'allegations',
            'violation', 'violations', 'breach', 'breaches', 'non-compliance',
            'illegal', 'unlawful', 'improper', 'questionable'
        }

    def _load_safety_words(self):
        # Words indicating safety/stability
        return {
            'stable', 'stability', 'secure', 'security', 'strong', 'strength',
            'robust', 'reliable', 'dependable', 'consistent', 'conservative',
            'prudent', 'cautious', 'protected', 'hedged', 'insured', 'guaranteed',
            'solid', 'sound', 'healthy', 'resilient', 'resilience'
        }

    def _load_certainty_words(self):
        # Words indicating certainty
        return {
            'certain', 'definite', 'definitely', 'clear', 'clearly', 'obvious',
            'obviously', 'undoubtedly', 'without doubt', 'guarantee', 'guaranteed',
            'assure', 'assured', 'confident', 'confidence', 'will', 'shall',
            'must', 'certainty', 'positive', 'positively'
        }

    def _load_uncertainty_words(self):
        # Words indicating uncertainty
        return {
            'uncertain', 'uncertainty', 'may', 'might', 'could', 'possibly',
            'perhaps', 'maybe', 'potentially', 'likely', 'unlikely', 'probability',
            'probable', 'estimate', 'estimated', 'approximately', 'roughly',
            'about', 'around', 'subject to', 'depending on', 'contingent'
        }

    def _load_litigious_words(self):
        # Words related to litigation
        return {
            'litigation', 'lawsuit', 'lawsuits', 'legal', 'proceeding', 'proceedings',
            'claim', 'claims', 'dispute', 'disputes', 'arbitration', 'settlement',
            'settlements', 'judgment', 'judgments', 'court', 'appeal', 'appeals',
            'plaintiff', 'defendant', 'sue', 'sued', 'class action', 'alleged'
        }

    def calculate_positivity(self, text):
        """Calculate positivity score"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        vader_scores = self.vader.polarity_scores(text)
        blob_scores = TextBlob(text).sentiment

        # Weighted average of multiple sentiment measures
        positivity = (vader_scores['pos'] * 0.6 +
                      (blob_scores.polarity + 1) / 2 * 0.4)

        return round(positivity, 4)

    def calculate_negativity(self, text):
        """Calculate negativity score"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        vader_scores = self.vader.polarity_scores(text)
        blob_scores = TextBlob(text).sentiment

        # Weighted average
        negativity = (vader_scores['neg'] * 0.6 +
                      (1 - blob_scores.polarity) / 2 * 0.4)

        return round(negativity, 4)

    def calculate_litigiousness(self, text):
        """Calculate litigiousness score based on legal word frequency"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        words = text.lower().split()
        if len(words) == 0:
            return 0.0

        # Count litigious words
        litigious_count = sum(1 for word in words if word in self.litigious_words)

        # Normalize by text length (words)
        score = litigious_count / len(words)
        return round(score, 4)

    def calculate_polarity(self, text):
        """Calculate overall polarity (-1 to 1)"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        vader_scores = self.vader.polarity_scores(text)
        blob_polarity = TextBlob(text).sentiment.polarity

        # Combine scores
        polarity = (vader_scores['compound'] * 0.5 + blob_polarity * 0.5)
        return round(polarity, 4)

    def calculate_risk(self, text):
        """Calculate risk score based on risk-related words"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        words = text.lower().split()
        if len(words) == 0:
            return 0.0

        # Count risk words
        risk_count = sum(1 for word in words if word in self.risk_words)

        # Normalize by text length
        score = risk_count / len(words)
        return round(score, 4)

    def calculate_readability(self, text):
        """Calculate readability score (higher = more readable)"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        try:
            # Flesch Reading Ease score (higher = easier to read)
            flesch_score = textstat.flesch_reading_ease(text)

            # Normalize to 0-1 scale (assuming typical range 0-100)
            normalized_score = min(max(flesch_score / 100, 0), 1)

            return round(normalized_score, 4)
        except:
            return 0.0

    def calculate_fraud(self, text):
        """Calculate fraud-related language score"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        words = text.lower().split()
        if len(words) == 0:
            return 0.0

        # Count fraud-related words
        fraud_count = sum(1 for word in words if word in self.fraud_words)

        # Normalize by text length
        score = fraud_count / len(words)
        return round(score, 4)

    def calculate_safety(self, text):
        """Calculate safety/stability score"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        words = text.lower().split()
        if len(words) == 0:
            return 0.0

        # Count safety words
        safety_count = sum(1 for word in words if word in self.safety_words)

        # Normalize by text length
        score = safety_count / len(words)
        return round(score, 4)

    def calculate_certainty(self, text):
        """Calculate certainty language score"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        words = text.lower().split()
        if len(words) == 0:
            return 0.0

        # Count certainty words
        certainty_count = sum(1 for word in words if word in self.certainty_words)

        # Normalize by text length
        score = certainty_count / len(words)
        return round(score, 4)

    def calculate_uncertainty(self, text):
        """Calculate uncertainty language score"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        words = text.lower().split()
        if len(words) == 0:
            return 0.0

        # Count uncertainty words
        uncertainty_count = sum(1 for word in words if word in self.uncertainty_words)

        # Normalize by text length
        score = uncertainty_count / len(words)
        return round(score, 4)

    def calculate_sentiment(self, text):
        """Calculate overall sentiment score (-1 to 1)"""
        if pd.isna(text) or text.strip() == "":
            return 0.0

        vader_scores = self.vader.polarity_scores(text)
        return round(vader_scores['compound'], 4)

    def analyze_all_scores(self, text):
        """Calculate all 11 scores for a given text"""
        if pd.isna(text) or text.strip() == "":
            return {score: 0.0 for score in self.get_score_names()}

        return {
            'positivity': self.calculate_positivity(text),
            'negativity': self.calculate_negativity(text),
            'litigiousness': self.calculate_litigiousness(text),
            'polarity': self.calculate_polarity(text),
            'risk': self.calculate_risk(text),
            'readability': self.calculate_readability(text),
            'fraud': self.calculate_fraud(text),
            'safety': self.calculate_safety(text),
            'certainty': self.calculate_certainty(text),
            'uncertainty': self.calculate_uncertainty(text),
            'sentiment': self.calculate_sentiment(text)
        }

    def get_score_names(self):
        """Return list of all score names"""
        return [
            'positivity', 'negativity', 'litigiousness', 'polarity',
            'risk', 'readability', 'fraud', 'safety', 'certainty',
            'uncertainty', 'sentiment'
        ]

# Main processing function
def compute_nlp_scores(df, text_column='md&a'):
    """
    Compute NLP scores for MD&A text column

    Parameters:
    -----------
    df : pandas.DataFrame
        Input dataframe containing text data
    text_column : str
        Name of the column containing text data

    Returns:
    --------
    pandas.DataFrame with new columns added
    """

    analyzer = FinancialTextAnalyzer()

    # Create progress bar for large datasets
    from tqdm import tqdm
    tqdm.pandas()

    print("Computing NLP scores...")

    # Calculate all scores
    scores = df[text_column].progress_apply(
        lambda x: analyzer.analyze_all_scores(x) if pd.notna(x) and str(x).strip() != ""
        else {score: np.nan for score in analyzer.get_score_names()}
    )

    # Convert to DataFrame
    scores_df = pd.DataFrame(scores.tolist(), index=df.index)

    # Add prefix to column names
    scores_df = scores_df.add_prefix('nlp_')

    # Concatenate with original dataframe
    result_df = pd.concat([df, scores_df], axis=1)

    print(f"Added {len(scores_df.columns)} NLP score columns:")
    for col in scores_df.columns:
        print(f"  - {col}")

    return result_df

# Optional: Enhanced version with more sophisticated measures
def compute_nlp_scores_enhanced(df, text_column='md&a'):
    """
    Enhanced version with additional preprocessing and validation
    """

    # Preprocess text function
    def preprocess_text(text):
        if pd.isna(text):
            return ""

        text = str(text)
        # Basic cleaning
        text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
        text = re.sub(r'[^\w\s.,;:!?\-$%]', '', text)  # Remove special chars
        text = text.strip()
        return text

    print("Preprocessing text...")
    df['md&a_processed'] = df[text_column].apply(preprocess_text)

    # Initialize analyzer
    analyzer = FinancialTextAnalyzer()

    # Calculate scores
    scores_data = []
    print("Computing enhanced NLP scores...")

    for idx, text in tqdm(enumerate(df['md&a_processed']), total=len(df)):
        if text and len(text.split()) > 10:  # Only process meaningful text
            scores = analyzer.analyze_all_scores(text)
        else:
            scores = {score: np.nan for score in analyzer.get_score_names()}
        scores_data.append(scores)

    # Create scores dataframe
    scores_df = pd.DataFrame(scores_data, index=df.index)
    scores_df = scores_df.add_prefix('nlp_')

    # Add additional derived metrics
    if 'nlp_positivity' in scores_df.columns and 'nlp_negativity' in scores_df.columns:
        scores_df['nlp_sentiment_balance'] = scores_df['nlp_positivity'] - scores_df['nlp_negativity']
        scores_df['nlp_risk_safety_ratio'] = scores_df['nlp_risk'] / (scores_df['nlp_safety'] + 0.001)  # Avoid division by zero

    # Combine with original
    result_df = pd.concat([df, scores_df], axis=1)

    # Drop temporary processed column if desired
    result_df = result_df.drop(columns=['md&a_processed'])

    return result_df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**Reasoning**:
The NLP functions have been successfully defined in the previous steps after correcting the import errors. Now, I will apply these functions to the 'md&a' column of the DataFrame to calculate the 11 NLP scores and add them as new columns.



In [10]:
df_with_nlp_scores = compute_nlp_scores(df.copy(), text_column='md&a')
display(df_with_nlp_scores.head())

Computing NLP scores...


100%|██████████| 2029/2029 [07:36<00:00,  4.44it/s]

Added 11 NLP score columns:
  - nlp_positivity
  - nlp_negativity
  - nlp_litigiousness
  - nlp_polarity
  - nlp_risk
  - nlp_readability
  - nlp_fraud
  - nlp_safety
  - nlp_certainty
  - nlp_uncertainty
  - nlp_sentiment


,Rating,Name,Ticker,Rating Agency Name,Date,Sector,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,...,nlp_negativity,nlp_litigiousness,nlp_polarity,nlp_risk,nlp_readability,nlp_fraud,nlp_safety,nlp_certainty,nlp_uncertainty,nlp_sentiment
0,A,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,11/27/2015,Consumer Durables,0.945894,0.426395,0.099690,44.203245,...,0.2210,0.0007,0.4995,0.0027,0.1746,0.0,0.0000,0.0000,0.0020,0.9988
1,BBB,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,2/13/2014,Consumer Durables,1.033559,0.498234,0.203120,38.991156,...,0.2077,0.0007,0.5118,0.0020,0.2130,0.0,0.0007,0.0007,0.0026,0.9992
2,BBB,Whirlpool Corporation,WHR,Fitch Ratings,3/6/2015,Consumer Durables,0.963703,0.451505,0.122099,50.841385,...,0.2148,0.0007,0.5018,0.0034,0.1711,0.0,0.0020,0.0027,0.0027,0.9994
3,BBB,Whirlpool Corporation,WHR,Fitch Ratings,6/15/2012,Consumer Durables,1.019851,0.510402,0.176116,41.161738,...,0.2161,0.0007,0.5090,0.0020,0.1239,0.0,0.0013,0.0020,0.0054,0.9995
4,BBB,Whirlpool Corporation,WHR,Standard & Poor's Ratings Services,10/24/2016,Consumer Durables,0.957844,0.495432,0.141608,47.761126,...,0.2143,0.0000,0.5021,0.0065,0.1597,0.0,0.0000,0.0032,0.0039,0.9975


**Reasoning**:
The previous step successfully computed and added the NLP scores to the DataFrame. To confirm the full structure of the updated DataFrame and verify the new columns, I will display the entire `df_with_nlp_scores` DataFrame.



In [11]:
display(df_with_nlp_scores)

,Rating,Name,Ticker,Rating Agency Name,Date,Sector,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,...,nlp_negativity,nlp_litigiousness,nlp_polarity,nlp_risk,nlp_readability,nlp_fraud,nlp_safety,nlp_certainty,nlp_uncertainty,nlp_sentiment
0,A,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,11/27/2015,Consumer Durables,0.945894,0.426395,0.099690,44.203245,...,0.2210,0.0007,0.4995,0.0027,0.1746,0.0000,0.0000,0.0000,0.0020,0.9988
1,BBB,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,2/13/2014,Consumer Durables,1.033559,0.498234,0.203120,38.991156,...,0.2077,0.0007,0.5118,0.0020,0.2130,0.0000,0.0007,0.0007,0.0026,0.9992
2,BBB,Whirlpool Corporation,WHR,Fitch Ratings,3/6/2015,Consumer Durables,0.963703,0.451505,0.122099,50.841385,...,0.2148,0.0007,0.5018,0.0034,0.1711,0.0000,0.0020,0.0027,0.0027,0.9994
3,BBB,Whirlpool Corporation,WHR,Fitch Ratings,6/15/2012,Consumer Durables,1.019851,0.510402,0.176116,41.161738,...,0.2161,0.0007,0.5090,0.0020,0.1239,0.0000,0.0013,0.0020,0.0054,0.9995
4,BBB,Whirlpool Corporation,WHR,Standard & Poor's Ratings Services,10/24/2016,Consumer Durables,0.957844,0.495432,0.141608,47.761126,...,0.2143,0.0000,0.5021,0.0065,0.1597,0.0000,0.0000,0.0032,0.0039,0.9975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,BBB,"NVR, Inc.",NVR,Moody's Investors Service,9/5/2012,Capital Goods,11.757767,7.115059,7.057420,1.079762,...,0.2290,0.0013,0.5034,0.0073,0.1810,0.0007,0.0020,0.0040,0.0113,0.9987
2025,BB,Kaiser Aluminum Corporation,KALU,Standard & Poor's Ratings Services,4/28/2016,Capital Goods,2.962788,1.294743,0.428234,30.602414,...,0.2168,0.0007,0.5176,0.0054,0.1387,0.0000,0.0034,0.0027,0.0054,0.9992
2026,B,Cresud S.A.C.I.F. y A.,CRESY,Fitch Ratings,11/30/2012,Finance,0.883875,0.842553,0.233830,147.599371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2027,B,Cresud S.A.C.I.F. y A.,CRESY,Fitch Ratings,6/15/2012,Finance,0.911713,0.748356,0.310640,131.644566,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Final Task

### Subtask:
Summarize the successful calculation and addition of all 11 NLP scores to the DataFrame.


## Summary:

### Data Analysis Key Findings
*   All 11 specified NLP scores (positivity, negativity, litigiousness, polarity, risk, readability, fraud, safety, certainty, uncertainty, and sentiment) were successfully calculated for the 'md&a' column across 2029 rows of the DataFrame.
*   The problematic import of `readability` was corrected, and necessary NLTK data (punkt, averaged\_perceptron\_tagger, wordnet) was downloaded, ensuring a proper setup for NLP processing.
*   The DataFrame `df` was successfully updated with 11 new columns, each containing one of the computed NLP scores, and the updated DataFrame (`df_with_nlp_scores`) was displayed, confirming the integration of these new metrics.
*   Missing values were appropriately handled during the NLP score calculation for entries where text data was not available.

### Insights or Next Steps
*   The newly added NLP scores provide a quantitative basis for analyzing the textual characteristics of the 'md&a' section, enabling deeper insights into the financial reports.
*   Further analysis can now be performed to correlate these NLP scores with other financial metrics or company performance indicators to uncover potential relationships.


In [15]:
df_with_nlp_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2029 entries, 0 to 2028
Data columns (total 48 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Rating                              2029 non-null   object 
 1   Name                                2029 non-null   object 
 2   Ticker                              2029 non-null   object 
 3   Rating Agency Name                  2029 non-null   object 
 4   Date                                2029 non-null   object 
 5   Sector                              2029 non-null   object 
 6   currentRatio                        2029 non-null   float64
 7   quickRatio                          2029 non-null   float64
 8   cashRatio                           2029 non-null   float64
 9   daysOfSalesOutstanding              2029 non-null   float64
 10  netProfitMargin                     2029 non-null   float64
 11  pretaxProfitMargin                  2029 no

In [22]:
df_with_nlp_scores['rating_date'] = pd.to_datetime(df_with_nlp_scores['rating_date'])
df_with_nlp_scores = df_with_nlp_scores.drop(columns=['Rating', 'Date'])
display(df_with_nlp_scores.head())

,Name,Ticker,Rating Agency Name,Sector,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,netProfitMargin,pretaxProfitMargin,...,nlp_negativity,nlp_litigiousness,nlp_polarity,nlp_risk,nlp_readability,nlp_fraud,nlp_safety,nlp_certainty,nlp_uncertainty,nlp_sentiment
0,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,Consumer Durables,0.945894,0.426395,0.099690,44.203245,0.037480,0.049351,...,0.2210,0.0007,0.4995,0.0027,0.1746,0.0,0.0000,0.0000,0.0020,0.9988
1,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,Consumer Durables,1.033559,0.498234,0.203120,38.991156,0.044062,0.048857,...,0.2077,0.0007,0.5118,0.0020,0.2130,0.0,0.0007,0.0007,0.0026,0.9992
2,Whirlpool Corporation,WHR,Fitch Ratings,Consumer Durables,0.963703,0.451505,0.122099,50.841385,0.032709,0.044334,...,0.2148,0.0007,0.5018,0.0034,0.1711,0.0,0.0020,0.0027,0.0027,0.9994
3,Whirlpool Corporation,WHR,Fitch Ratings,Consumer Durables,1.019851,0.510402,0.176116,41.161738,0.020894,-0.012858,...,0.2161,0.0007,0.5090,0.0020,0.1239,0.0,0.0013,0.0020,0.0054,0.9995
4,Whirlpool Corporation,WHR,Standard & Poor's Ratings Services,Consumer Durables,0.957844,0.495432,0.141608,47.761126,0.042861,0.053770,...,0.2143,0.0000,0.5021,0.0065,0.1597,0.0,0.0000,0.0032,0.0039,0.9975


In [23]:
df_with_nlp_scores.to_csv('credit_ratings_multimodal.csv', index=False)
print("DataFrame saved to 'credit_ratings_multimodal.csv'")

DataFrame saved to 'credit_ratings_multimodal.csv'


In [24]:
df_with_nlp_scores['Sector'].value_counts()

,count
Sector,
Energy,294
Basic Industries,260
Consumer Services,250
Technology,234
Capital Goods,233
Public Utilities,211
Health Care,171
Consumer Non-Durables,132
Consumer Durables,74
